Let's cleanse data and construct the model base on the results of EDA.

First, I will create a baseline model. It puts data right into the model, which is LinearRegression.

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns

from sklearn.linear_model import LinearRegression
# from sklearn.preprocessing import TargetEncoder
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

import copy

train_raw = pd.read_csv('train.csv')
test_raw = pd.read_csv('test.csv')

In [47]:
def cleanse_data(train_set, test_set, test_has_label=True):

    

    # 1. delete 'brand'
    train_set = train_set.drop(['brand'], axis=1)
    test_set = test_set.drop(['brand'], axis=1)

    # 3. drop ext_col, int_col
    train_set = train_set.drop(['ext_col', 'int_col'], axis=1)
    test_set = test_set.drop(['ext_col', 'int_col'], axis=1)

    # 4. drop id
    train_set = train_set.drop(['id'], axis=1)
    test_set = test_set.drop(['id'], axis=1)

    ''' only 8 features ('model_year', 'milage', 'fuel_type', 'accident', 'clean_title', 'model', 'engine') left '''

    # 5. for the same model value group, delete all rows that has high price (above 90% quantile).
    quantiles = train_set.groupby('model')['price'].quantile(0.9)
    train_set = train_set[train_set.apply(lambda row: row['price'] <= quantiles[row['model']], axis=1)]
    
    # 6. Filter out rows that has price higher than 500K
    train_set = train_set[train_set.apply(lambda row: row['price'] < 500000, axis=1)]

    # 7. convert 'year' to int, and calculate year passed from min(year)
    base = min(train_set['model_year'])
    train_set['model_year'] = (train_set['model_year'].astype(int) - base)
    test_set['model_year']  = (test_set['model_year'] .astype(int) - base)
    
    # 9. Aggregate categorical features in transmission
    col_names = [
    'A/T',
    'Transmission w/Dual Shift Mode',
    '7-Speed A/T',
    '8-Speed A/T',
    '10-Speed Automatic',
    '1-Speed A/T',
    '6-Speed A/T',
    '10-Speed A/T',
    '9-Speed A/T',
    '8-Speed Automatic',
    '9-Speed Automatic',
    '5-Speed A/T',
    'Automatic',
    '7-Speed Automatic with Auto-Shift',
    'CVT Transmission',
    '5-Speed M/T',
    'M/T',
    '6-Speed M/T',
    '6-Speed Automatic',
    '4-Speed Automatic',
    '7-Speed M/T',
    '2-Speed A/T',
    '1-Speed Automatic',
    'Automatic CVT',
    '4-Speed A/T',
    '6-Speed Manual',
    'Transmission Overdrive Switch',
    '8-Speed Automatic with Auto-Shift',
    '7-Speed Manual',
    '7-Speed Automatic',
    '9-Speed Automatic with Auto-Shift',
    '6-Speed Automatic with Auto-Shift',
    '6-Speed Electronically Controlled Automatic with O',
    'F',
    'CVT-F',
    '8-Speed Manual',
    'Manual',
    '-',
    '2',
    '6 Speed At/Mt',
    '5-Speed Automatic',
    '2-Speed Automatic',
    '8-SPEED A/T',
    '7-Speed',
    'Variable',
    'Single-Speed Fixed Gear',
    '8-SPEED AT',
    '10-Speed Automatic with Overdrive',
    '7-Speed DCT Automatic',
    'SCHEDULED FOR OR IN PRODUCTION',
    '6-Speed',
    '6 Speed Mt'
]
    
    col_names_override = [
    'A/T',
    'Transmission w/Dual Shift Mode',
    '7-Speed A/T',
    '8-Speed A/T',
    '10-Speed A/T',
    '1-Speed A/T',
    '6-Speed A/T',
    '10-Speed A/T',
    '9-Speed A/T',
    '8-Speed A/T',
    '9-Speed A/T',
    '5-Speed A/T',
    'A/T',
    '7-Speed A/T with Auto-Shift',
    'CVT Transmission',
    '5-Speed M/T',
    'M/T',
    '6-Speed M/T',
    '6-Speed A/T',
    '4-Speed A/T',
    '7-Speed M/T',
    '2-Speed A/T',
    '1-Speed A/T',
    'A/T CVT',
    '4-Speed A/T',
    '6-SpeedM/T',
    'Transmission Overdrive Switch',
    '8-Speed A/T with Auto-Shift',
    '7-Speed M/T',
    '7-Speed A/T',
    '9-Speed A/T with Auto-Shift',
    '6-Speed A/T with Auto-Shift',
    '6-Speed Electronically Controlled A/T with O',
    '-',
    'CVT-F',
    '8-Speed M/T',
    'M/T',
    '-',
    '-',
    '6 Speed At/Mt',
    '5-Speed A/T',
    '2-Speed A/T',
    '8-Speed A/T',
    '7-Speed',
    'Variable',
    'Single-Speed Fixed Gear',
    '8-Speed AT',
    '10-Speed A/T with Overdrive',
    '7-Speed DCT A/T',
    '-',
    '6-Speed',
    '6 Speed M/T'
]
    
    trans_dict = dict(zip(col_names, col_names_override))
    train_set['transmission'] = train_set['transmission'].replace(trans_dict)
    test_set['transmission']  = test_set['transmission'].replace(trans_dict)

    # Get all numeric values
    # I will add categoricals later
    

    
    categoricals = ['accident', 'clean_title']#['transmission', 'fuel_type', 'accident', 'clean_title', 'model', 'engine']


    data_average = train_set.groupby('model')['price'].mean().reset_index()
    # print(data_average.shape)
    data_average.columns = ['model', 'average_price']
    print(data_average.head())
    print(train_set.head())
    replace_dict = dict(zip(data_average['model'], data_average['average_price']))
    train_set['model'] = train_set['model'].replace(replace_dict)
    print(train_set.head())
    
    # for the test data too:
    data_average = train_set.groupby('model')['price'].mean().reset_index()
    # print(data_average.shape)
    data_average.columns = ['model', 'average_price']
    test_set['model'] = test_set['model'].replace(replace_dict)
    test_set['model'] = test_set['model'].apply(lambda x: 20000 if isinstance(x, str) else x)

    train_processed = train_set.loc[:, ('model', 'milage', 'model_year', 'price')]
    if test_has_label:
        test_processed = test_set.loc[:, ('model', 'milage', 'model_year', 'price')]
    else:
        test_processed = test_set.loc[:, ('model', 'milage', 'model_year')]

    # print(train_set.columns)

    for i in categoricals:

        train_set_part = train_set[i]
        test_set_part = test_set[i]

        train_set_part = train_set_part.fillna('blank')
        test_set_part  = test_set_part.fillna('blank')

        # set values of train_set[i] to 'blank', 1% of them
        train_set_part.loc[train_set.sample(frac=0.01).index] = 'blank'

        # Get all unique values in train_set
        train_value_set = set(train_set_part)

        # get_dummies for train's model
        train_set_encoded = pd.get_dummies(train_set_part, prefix=i)

        # Remove categories that are not in train, switch them to 'blank'
        test_set[i] = test_set[i].apply(lambda x: 'blank' if x not in train_value_set else x)
        test_set_encoded = pd.get_dummies(test_set[i], prefix=i)

        # Add columns for train set
        for j in train_set_encoded.columns:
            if j not in test_set_encoded.columns:
                test_set_encoded[j] = 0
        
        # Add blank columns if it was not created
        if i+'_blank' not in train_set_encoded.columns:
            train_set_encoded[i+'_blank'] = 0
        if i+'_blank' not in test_set_encoded.columns:
            test_set_encoded[i+'_blank'] = 0
        
        train_processed = pd.concat((train_processed, train_set_encoded), axis=1)
        test_processed  = pd.concat((test_processed,  test_set_encoded), axis=1)

    # sort columns
    train_processed = train_processed[train_processed.columns.sort_values()]
    test_processed  = test_processed[test_processed.columns.sort_values()]

    print(train_processed.head())
    

    return train_processed, test_processed

In [44]:
'''
Refactoring of the function above...
'''
import copy

def cleanse_data(train_set, test_set, test_has_label=True):

    train_set = copy.deepcopy(train_set)
    test_set = copy.deepcopy(test_set)
    
    # 1. delete 'brand'. This data is incorrect so often.
    train_set = train_set.drop(['brand'], axis=1)
    test_set = test_set.drop(['brand'], axis=1)

    # 3. drop ext_col, int_col
    train_set = train_set.drop(['ext_col', 'int_col'], axis=1)
    test_set = test_set.drop(['ext_col', 'int_col'], axis=1)

    # # 4. drop id
    # train_set = train_set.drop(['id'], axis=1)
    # test_set = test_set.drop(['id'], axis=1)

    ''' only 8 features ('model_year', 'milage', 'fuel_type', 'accident', 'clean_title', 'model', 'engine') left '''

    # 5. for the same model value group, delete all rows that has high price (above 90% quantile).
    quantiles = train_set.groupby('model')['price'].quantile(0.9)
    train_set = train_set[train_set.apply(lambda row: row['price'] <= quantiles[row['model']], axis=1)]
    
    # 6. Filter out rows that has price higher than 500K
    train_set = train_set[train_set.apply(lambda row: row['price'] < 500000, axis=1)]

    # 7. convert 'year' to int, and calculate year passed from min(year)
    base = min(train_set['model_year'])
    train_set['model_year'] = (train_set['model_year'].astype(int) - base)**2
    test_set['model_year']  = (test_set['model_year'] .astype(int) - base)**2

    # convert model name to the average of price
    data_average = train_set.groupby('model')['price'].mean().reset_index()
    data_average.columns = ['model', 'average_price']
    replace_dict = dict(zip(data_average['model'], data_average['average_price']))

    train_set['model'] = train_set['model'].replace(replace_dict)
    test_set['model'] = test_set['model'].replace(replace_dict)

    # If there is a model that was not in train set, set it to 20000
    test_set['model'] = test_set['model'].apply(lambda x: 20000 if isinstance(x, str) else x)

    # Change accident value
    train_set.loc[train_set.sample(frac=0.01).index, 'accident'] = 'blank'
    replace_dict = dict(zip(['At least 1 accident or damage reported', 'blank', 'None reported'], [1, 1, 0]))
    train_set['accident'] = train_set['accident'].replace(replace_dict)
    train_set['accident'].fillna(1, inplace=True)
    test_set['accident'] = test_set['accident'].replace(replace_dict)
    train_set['accident'].fillna(1, inplace=True)

    # Change clean_title value
    train_set.loc[train_set.sample(frac=0.01).index, 'clean_title'] = 'blank'
    replace_dict = dict(zip(['Yes', 'blank', None], [1, 0, 0]))
    train_set['clean_title'] = train_set['clean_title'].replace(replace_dict)
    train_set['clean_title'].fillna(0, inplace=True)
    test_set['clean_title'] = test_set['clean_title'].replace(replace_dict)
    test_set['clean_title'].fillna(0, inplace=True)

    train_set.drop(['engine', 'transmission', 'fuel_type'], axis=1, inplace=True)
    test_set.drop(['engine', 'transmission', 'fuel_type'], axis=1, inplace=True)

    for i in ['model', 'milage', 'model_year']:
        scaler = StandardScaler()
        train_set[i] = scaler.fit_transform(train_set[i].values.reshape(-1, 1))
        test_set[i]  = scaler.transform(test_set[i].values.reshape(-1, 1))

    return train_set, test_set


In [36]:
def preprocess(train, test):

    train = copy.deepcopy(train)
    test = copy.deepcopy(test)

    print(train.columns)

    train = train.drop(columns='ext_col', axis=1)
    train = train.drop(columns='int_col', axis=1)
    train = train.drop(columns='brand', axis=1)
    train = train.drop(columns='engine', axis=1)
    train = train.drop(columns='transmission', axis=1)

    test = test.drop(columns='ext_col', axis=1)
    test = test.drop(columns='int_col', axis=1)
    test = test.drop(columns='brand', axis=1)
    test = test.drop(columns='engine', axis=1)
    test = test.drop(columns='transmission', axis=1)

    # Filter out rows that has price higher than 90% quantile.
    quantiles = train.groupby('model')['price'].quantile(0.9)
    train = train[train.apply(lambda row: row['price'] <= quantiles[row['model']], axis=1)].reset_index(drop=True)
    
    # 6. Filter out rows that has price higher than 500K
    train = train[train.apply(lambda row: row['price'] < 500000, axis=1)].reset_index(drop=True)
    
    TRAIN_LN = int(len(train)*0.8)
    CATS = [c for c in train.columns if not c in ["id","price"] ]
    NUMS = ['milage']
    CATS = [c for c in CATS if not c in NUMS]
    print("Categorical features:", CATS )
    print("Numerical features:", NUMS)
    print("STANDARDIZING: ",end="")
    for c in NUMS:
        print(c,', ',end='')
        m = train[c].mean()
        s = train[c].std()
        train[c] = (train[c]-m)/s
        train[c] = train[c].fillna(m)
    CAT_SIZE = []
    CAT_EMB = []
    RARE = []

    print("LABEL ENCODING:")
    for c in CATS:
        # LABEL ENCODE
        train[c],_ = train[c].factorize()
        # train[c] -= train[c].min()
        vc = train[c].value_counts()

        test[c],_ = test[c].factorize()
        # test[c] -= test[c].min()
        
        # IDENTIFY RARE VALUES
        RARE.append( vc.loc[vc<40].index.values )
        n = train[c].nunique()
        mn = train[c].min()
        mx = train[c].max()
        r = len(RARE[-1])
        print(f'{c}: nunique={n}, min={mn}, max={mx}, rare_ct={r}')
        
        # RELABEL RARE VALUES AS ZERO
        CAT_SIZE.append(mx+1 +1) #ADD ONE FOR RARE
        CAT_EMB.append( int(np.ceil( np.sqrt(mx+1 +1))) ) # ADD ONE FOR RARE
        train[c] += 1
        test[c] += 1
        print(train[c].isin(RARE[-1]))
        train.loc[train[c].isin(RARE[-1]),c] = 0
        test.loc[train[c].isin(RARE[-1]),c] = 0

    # test = train.iloc[TRAIN_LN:]
    # train = train.iloc[:TRAIN_LN]
    
    for c in CATS:
        # COMPARE TEST CAT VALUES TO TRAIN CAT VALUES
        A = train[c].unique()
        B = test[c].unique()
        print(A, B)
        C = np.setdiff1d(B,A)
        print(f"{c}: Test has label encodes = {C} which are not in train.")
        if len(C)>0:
            print(f" => {len(test.loc[test[c].isin(C)])} rows" )
            
        # RELABEL UNSEEN TEST VALUES AS ZERO
        test.loc[test[c].isin(C),c] = 0 

    return train, test

In [49]:
# sns.boxplot(train['price'])
# plt.show()

# train_test_split, test is 0.2
# test_raw = test_
# train_raw = train_raw[:int(len(train_raw)*0.8)]
train_data, test_data = cleanse_data(train_raw, test_raw, test_has_label=False)
# summarize dataframe

train_data.to_csv('C:/Projects/Kaggle/Used Car Prices/train_data.csv', index=False)
print(train_data.shape)
print(test_data.shape)

C:\Users\diamo\AppData\Local\Temp\ipykernel_10728\3754565328.py:42: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_set['model'] = train_set['model'].replace(replace_dict)
C:\Users\diamo\AppData\Local\Temp\ipykernel_10728\3754565328.py:51: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_set['accident'] = train_set['accident'].replace(replace_dict)
C:\Users\diamo\AppData\Local\Temp\ipykernel_10728\3754565328.py:52: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using

(169224, 7)
(125690, 6)


In [53]:
from autogluon.tabular import TabularDataset, TabularPredictor

# test_id = test_data['id']

if type(train_data) is not TabularDataset:
    train_data = TabularDataset(train_data.drop(columns='id'))
    test_data = TabularDataset(test_data.drop(columns='id'))
    
TRAIN_LN = int(0.8*len(train_data))
valid_data = train_data[TRAIN_LN:]
train_data = train_data[:TRAIN_LN]

params = {
	'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}, {}, 'GBMLarge'],
	'CAT': {},
	'XGB': {},
	'RF': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
	'XT': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
	'KNN': [{'weights': 'uniform', 'ag_args': {'name_suffix': 'Unif'}}, {'weights': 'distance', 'ag_args': {'name_suffix': 'Dist'}}],
}
predictor = TabularPredictor(label='price', problem_type='regression').fit(train_data=train_data, hyperparameters=params)
predictions = predictor.predict(test_data)

No path specified. Models will be saved in: "AutogluonModels\ag-20240911_022650"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.7
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          8
Memory Avail:       5.98 GB / 15.85 GB (37.8%)
Disk Space Avail:   355.89 GB / 475.76 GB (74.8%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial proto

In [56]:
# predictor = TabularPredictor.load("AutogluonModels/ag-20240911_021401")
# predictor.evaluate(test_data)
# predictor.leaderboard(test_data)

# RMSE

from sklearn.metrics import mean_squared_error

test_id = test_data['id']

print(test_id)
print(predictions)

result = pd.concat((test_id, predictions), axis=1)

result.to_csv('result.csv', index=False)
# print('RMSE:', mean_squared_error(test_data, predictions, squared=False)) # mean_squared_error(test_data['price'], prediction, squared=False)

0         188533
1         188534
2         188535
3         188536
4         188537
           ...  
125685    314218
125686    314219
125687    314220
125688    314221
125689    314222
Name: id, Length: 125690, dtype: int64
0         16587.445312
1         67183.015625
2         46520.953125
3         25980.625000
4         29084.169922
              ...     
125685    19927.986328
125686    44360.398438
125687    20882.740234
125688    16210.500000
125689    34473.070312
Name: price, Length: 125690, dtype: float32


c:\Users\diamo\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


ValueError: Input contains NaN.

In [53]:
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge

# add sklearn Ridge

params = {
    'objective': 'regression',  # Multiclass classification task
    'metric': 'rmse',  # Logarithmic Loss as the evaluation metric for multiclass classification
    'boosting_type': 'gbdt',
    # 'early_stopping_rounds': 10,
    'max_depth': 10, 
    'lambda_l1': 0.2, 
    'lambda_l2': 0.2, 
    'min_data_in_leaf': 20, 
    'min_gain_to_split': 0.01, 
    'feature_fraction': 0.5, 
    'bagging_fraction': 0.5,
    'verbosity': -1
}

model = LGBMRegressor(**params)

model.fit(train_data.drop('price', axis=1), train_data['price'])

print('R^2:', model.score(train_data.drop('price', axis=1), train_data['price']))

print('RMSE:', np.sqrt(mean_squared_error(test_data['price'], model.predict(test_data.drop('price', axis=1)))))

model2 = Ridge()

model2.fit(train_data.drop('price', axis=1), train_data['price'])

print('R^2:', model2.score(train_data.drop('price', axis=1), train_data['price']))

print('RMSE:', np.sqrt(mean_squared_error(test_data['price'], (model2.predict(test_data.drop('price', axis=1))))))

R^2: 0.6586062836180284
RMSE: 76249.12764291337
R^2: 0.5908319593168465
RMSE: 76814.7313625287


In [5]:

print('RMSE:', np.sqrt(mean_squared_error(test_data['price'], model.predict(test_data.drop('price', axis=1)))))

RMSE: 77126.57351416007


In [56]:
import pandas as pd

# Sample DataFrames
train_df = pd.DataFrame({
    'model_engine': ['A', 'B', 'C', 'A', 'B'],
    'other_feature': [1, 2, 3, 4, 5]
})

test_df = pd.DataFrame({
    'model_engine': ['A', 'C', 'D', 'E'],
    'other_feature': [6, 7, 8, 9]
})

# Get unique model_engine values from train set
train_engines = set(train_df['model_engine'])

# One-hot encode the 'model_engine' feature in the training set
train_encoded = pd.get_dummies(train_df['model_engine'], prefix='engine')

# Add the encoded columns to the train_df
train_df = pd.concat([train_df, train_encoded], axis=1)

# One-hot encode the 'model_engine' feature in the test set
test_encoded = pd.get_dummies(test_df['model_engine'], prefix='engine')

# Add the encoded columns to the test_df
test_df = pd.concat([test_df, test_encoded], axis=1)

# Add 'model_engine_blank' column to test_df
test_df['model_engine_blank'] = test_df['model_engine'].apply(lambda x: 1 if x not in train_engines else 0)

# Ensure test_df has the same columns as train_df (excluding 'model_engine_blank')
for column in train_encoded.columns:
    if column not in test_df.columns:
        test_df[column] = 0

# Add 'model_engine_blank' column to train_df (all zeros)
train_df['model_engine_blank'] = 0

# Ensure the order of columns in test_df matches train_df
test_df = test_df[train_df.columns]

# Fill missing columns in test_df with zeros
test_df.fillna(0, inplace=True)

# Display the final DataFrames
print("Train DataFrame:")
print(train_df)
print("\nTest DataFrame:")
print(test_df)

Train DataFrame:
  model_engine  other_feature  engine_A  engine_B  engine_C  \
0            A              1         1         0         0   
1            B              2         0         1         0   
2            C              3         0         0         1   
3            A              4         1         0         0   
4            B              5         0         1         0   

   model_engine_blank  
0                   0  
1                   0  
2                   0  
3                   0  
4                   0  

Test DataFrame:
  model_engine  other_feature  engine_A  engine_B  engine_C  \
0            A              6         1         0         0   
1            C              7         0         0         1   
2            D              8         0         0         0   
3            E              9         0         0         0   

   model_engine_blank  
0                   0  
1                   0  
2                   1  
3                   1  
